In [8]:
import cv2
import torch
import easyocr
import pytesseract
import pandas as pd
from ultralytics import YOLO
import time

In [9]:
video_input = "test.mp4"
video_output = "resultado.mp4"
csv_output = "resultados.csv"

# Ruta al ejecutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Inicializar OCR
reader_easy = easyocr.Reader(['es'], gpu=True)

# Modelos YOLO
modelo_vehiculos = YOLO("yolo11n.pt").to('cuda') 
modelo_matriculas = YOLO("runs/detect/train8/weights/best.pt").to('cuda')  
# Video capture
cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    print("Error: no se puede abrir el video")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter para guardar resultados
out = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

resultados = []
frame_id = 0

# ----------------------------
# Procesamiento con tracking
# ----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección y tracking
    pred = modelo_vehiculos.track(frame, persist=True)[0]

    for obj in pred.boxes:
        x1, y1, x2, y2 = map(int, obj.xyxy[0])
        tipo = modelo_vehiculos.names[int(obj.cls[0])]
        conf = float(obj.conf[0])
        track_id = int(obj.id[0]) if hasattr(obj, 'id') else -1

        matricula_easy = ""
        matricula_tesseract = ""
        time_easy = 0
        time_tesseract = 0

        # Dibujar caja del vehículo
        color = (0, 255, 0) if tipo.lower() == "person" else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame,f"{tipo} ID:{track_id}",(x1,y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
        
        # Solo coches tienen matrícula
        if tipo.lower() == "car":
            roi_car = frame[y1:y2, x1:x2]
            if roi_car.size > 0:
                pred_matricula = modelo_matriculas.predict(roi_car, conf=0.3)
                matricula_roi = None
                if pred_matricula:
                    for pm in pred_matricula:
                        if pm.boxes:
                            for bbox, conf_m in zip(pm.boxes.xyxy, pm.boxes.conf):
                                if conf_m > 0.4:
                                    mx1, my1, mx2, my2 = map(int, bbox)
                                    matricula_roi = roi_car[my1:my2, mx1:mx2]
                                    break
                        if matricula_roi is not None:
                            break

                if matricula_roi is not None:
                    # Preprocesamiento para OCR
                    roi_gray = cv2.cvtColor(matricula_roi, cv2.COLOR_BGR2GRAY)
                    roi_gray = cv2.resize(roi_gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                    _, roi_thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                    # EasyOCR
                    start_easy = time.time()
                    res_easy = reader_easy.readtext(roi_thresh)
                    end_easy = time.time()
                    if res_easy:
                        best_text = ""
                        best_conf = 0
                        for (bbox_e, text, conf_e) in res_easy:
                            if conf_e > best_conf and len(text) >= 4:
                                best_text = text
                                best_conf = conf_e
                        matricula_easy = best_text
                    time_easy = end_easy - start_easy

                    # Tesseract
                    start_tess = time.time()
                    custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                    matricula_tesseract = pytesseract.image_to_string(roi_thresh, config=custom_config).strip()
                    end_tess = time.time()
                    time_tesseract = end_tess - start_tess

                    # Dibujar matrícula detectada
                    cv2.rectangle(frame, (x1+mx1, y1+my1), (x1+mx2, y1+my2), (0,0,255), 2)
                    cv2.putText(frame, f"{matricula_tesseract}", (x1+mx1, y1+my1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

        # Guardar resultados
        resultados.append({
            "frame": frame_id,
            "tipo": tipo,
            "conf": conf,
            "id": track_id,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "matricula_easy": matricula_easy,
            "matricula_tesseract": matricula_tesseract,
            "time_easy": time_easy,
            "time_tesseract": time_tesseract
        })

    # Escribir frame
    out.write(frame)
    frame_id += 1

# Guardar CSV
df = pd.DataFrame(resultados)
df.to_csv(csv_output, index=False, sep=";")

# Liberar recursos
cap.release()
out.release()
print("Finalizado. CSV y vídeo generado.")



0: 384x640 2 persons, 14 cars, 23.1ms
Speed: 8.2ms preprocess, 23.1ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 license_plate, 9.0ms
Speed: 2.2ms preprocess, 9.0ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 (no detections), 18.7ms
Speed: 3.0ms preprocess, 18.7ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 640)

0: 448x640 (no detections), 7.9ms
Speed: 1.5ms preprocess, 7.9ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 (no detections), 9.4ms
Speed: 1.9ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 (no detections), 10.1ms
Speed: 1.8ms preprocess, 10.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 640x576 (no detections), 14.3ms
Speed: 3.7ms preprocess, 14.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 576)

0: 480x640 (no detections), 8.6ms
Speed: 1.0ms preprocess, 8.6ms infere